In [6]:
import pandas as pd
import numpy as np
from sklearn import *


/anaconda3/lib/python3.6/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)
/anaconda3/lib/python3.6/site-packages/sklearn/ensemble/weight_boosting.py:29: DeprecationWarning: numpy.core.umath_tests is an internal NumPy module and should not be imported. It will be removed in a future NumPy release.
  from numpy.core.umath_tests import inner1d
/anaconda3/lib/python3.6/site-packages/sklearn/grid_search.py:42: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. This module will be removed in 0.20.
  DeprecationWarning)
/anaconda3/li

In [10]:
df = pd.read_csv("/data/kaggle/data_combined_cleaned.csv")
df = df[~df.SalesPrice.isnull()]
del df["Id"]
df.head()

,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,LotConfig,LandSlope,...,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalesPrice
0,60,RL,65.0,8450,Pave,None,Reg,Lvl,Inside,Gtl,...,0,None,None,None,0,2,2008,WD,Normal,208500.0
1,20,RL,80.0,9600,Pave,None,Reg,Lvl,FR2,Gtl,...,0,None,None,None,0,5,2007,WD,Normal,181500.0
2,60,RL,68.0,11250,Pave,None,IR1,Lvl,Inside,Gtl,...,0,None,None,None,0,9,2008,WD,Normal,223500.0
3,70,RL,60.0,9550,Pave,None,IR1,Lvl,Corner,Gtl,...,0,None,None,None,0,2,2006,WD,Abnorml,140000.0
4,60,RL,84.0,14260,Pave,None,IR1,Lvl,FR2,Gtl,...,0,None,None,None,0,12,2008,WD,Normal,250000.0


In [28]:
label = "SalesPrice"
X = df.drop(columns=[label]) # features
y = np.log(df[label])

X_dummy = pd.get_dummies(X, drop_first=True) # OneHot Encoding



#X: features, y: label
X_train, X_test, y_train, y_test = model_selection.train_test_split(X_dummy,
                                         y, test_size = 0.3, random_state = 1546)


pipe = pipeline.Pipeline([
    ("poly", preprocessing.PolynomialFeatures(degree=1, include_bias=False)),
    ("scaler", preprocessing.StandardScaler()),
    ("est", linear_model.Lasso(alpha=0.01, max_iter= 1000))
])
pipe.fit(X_train, y_train)

y_test_pred = pipe.predict(X_test)
y_train_pred = pipe.predict(X_train)

print("Train RMSE", np.sqrt(metrics.mean_squared_error(y_train, y_train_pred)))
print("Test RMSE", np.sqrt(metrics.mean_squared_error(y_test, y_test_pred)))
print("Train R2", metrics.r2_score(y_train, y_train_pred))
print("Test R2", metrics.r2_score(y_test, y_test_pred))

Train RMSE 0.12588682835163964
Test RMSE 0.15429986992477038
Train R2 0.9038828795327387
Test R2 0.8370017674943548


In [20]:
scores = model_selection.cross_val_score(pipe, X_train, y_train
                        , cv = 5, scoring="neg_mean_squared_error") # k = 5

In [23]:
np.sqrt(- scores)

array([0.11699036, 0.13593565, 0.22748628, 0.11001659, 0.14806249])

In [25]:
- np.mean(scores)

0.02358828115255892

In [37]:
0.01 + (np.random.random(10) - 0.5) * 0.001

array([0.0097385 , 0.01038867, 0.00992258, 0.00967589, 0.01003171,
       0.01005233, 0.00954327, 0.00967388, 0.00987551, 0.01039339])

In [39]:
param_grid = {
    #"poly__degree": [1, 2],
    "est__alpha": 0.01 + (np.random.random(10) - 0.5) * 0.001
}

gsearch = model_selection.GridSearchCV(pipe, param_grid=param_grid, scoring="r2")
gsearch.fit(X_train, y_train)

GridSearchCV(cv=None, error_score='raise',
       estimator=Pipeline(memory=None,
     steps=[('poly', PolynomialFeatures(degree=1, include_bias=False, interaction_only=False)), ('scaler', StandardScaler(copy=True, with_mean=True, with_std=True)), ('est', Lasso(alpha=0.01, copy_X=True, fit_intercept=True, max_iter=1000,
   normalize=False, positive=False, precompute=False, random_state=None,
   selection='cyclic', tol=0.0001, warm_start=False))]),
       fit_params=None, iid=True, n_jobs=1,
       param_grid={'est__alpha': array([0.01026, 0.01032, 0.00956, 0.00981, 0.01043, 0.0099 , 0.01016,
       0.00956, 0.01013, 0.00996])},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring='r2', verbose=0)

In [40]:
gsearch.best_params_

{'est__alpha': 0.009557372148947793}

In [41]:
gsearch.score(X_test, y_test) # gsearch.scoring

0.8378036448444643

In [43]:
label = "SalesPrice"
X = df.drop(columns=[label]) # features
y = np.log(df[label])

X_dummy = pd.get_dummies(X, drop_first=True) # OneHot Encoding



#X: features, y: label
X_train, X_test, y_train, y_test = model_selection.train_test_split(X_dummy,
                                         y, test_size = 0.3, random_state = 1546)


pipe = pipeline.Pipeline([
    ("poly", preprocessing.PolynomialFeatures(degree=1, include_bias=False)),
    ("scaler", preprocessing.StandardScaler()),
    ("est", linear_model.Ridge(alpha=0.01, max_iter= 1000))
])

param_grid = {
    #"poly__degree": [1, 2],
    "est__alpha": 0.01 + (np.random.random(10) - 0.5) * 0.001
}

gsearch = model_selection.GridSearchCV(pipe, param_grid=param_grid, scoring="r2")
gsearch.fit(X_train, y_train)

y_test_pred = gsearch.predict(X_test)
y_train_pred = gsearch.predict(X_train)

print("Train RMSE", np.sqrt(metrics.mean_squared_error(y_train, y_train_pred)))
print("Test RMSE", np.sqrt(metrics.mean_squared_error(y_test, y_test_pred)))
print("Train R2", metrics.r2_score(y_train, y_train_pred))
print("Test R2", metrics.r2_score(y_test, y_test_pred))

Train RMSE 0.08369728811232494
Test RMSE 0.15247112686176573
Train R2 0.9575123274935279
Test R2 0.840842541594902
